In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_97920694e1374eeb8eab9b0e1dbbc48c.csv"
df=pd.read_csv(url,header=None,na_values=["?"])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,1,1.0,NaN,3.0,2,2,1,2,2,2,2,2.0,2,2,2.0,2,2
1,1,1,1.0,NaN,3.0,2,2,2,2,2,1,2,2.0,2,1,2.0,1,2
2,1,1,2.0,2.0,3.0,1,2,2,2,2,2,2,2.0,2,2,2.0,1,2
3,1,1,2.0,NaN,3.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2
4,1,1,2.0,NaN,3.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2


In [3]:
df.shape

(339, 18)

In [4]:
df.isna().any()

0     False
1     False
2      True
3      True
4      True
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12     True
13    False
14    False
15     True
16    False
17    False
dtype: bool

In [5]:
df.isna().sum()/len(df)

0     0.000000
1     0.000000
2     0.002950
3     0.197640
4     0.457227
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.002950
13    0.000000
14    0.000000
15    0.002950
16    0.000000
17    0.000000
dtype: float64

#### Drop columns 3 and 4. These two columns consist of too many empty cells.

In [6]:
df=df.drop(columns=[3,4])
df.head()

,0,1,2,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,1,1.0,2,2,1,2,2,2,2,2.0,2,2,2.0,2,2
1,1,1,1.0,2,2,2,2,2,1,2,2.0,2,1,2.0,1,2
2,1,1,2.0,1,2,2,2,2,2,2,2.0,2,2,2.0,1,2
3,1,1,2.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2
4,1,1,2.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2


In [7]:
df.iloc[:,[2,10,13]].mode()

,2,12,15
0,2.0,2.0,2.0


In [8]:
df=df.fillna(2)
df.isna().any()

0     False
1     False
2     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
dtype: bool

In [9]:
df=df.rename(columns={0:"Class"})
y=df[["Class"]]
y.groupby("Class")["Class"].count()

Class
1     84
2     20
3      9
4     14
5     39
6      1
7     14
8      6
10     2
11    28
12    16
13     7
14    24
15     2
16     1
17    10
18    29
19     6
20     2
21     1
22    24
Name: Class, dtype: int64

#### Drop the classes with only 1 or 2 instances. These classes will not receive equal representation in training and test subsets.

In [10]:
sub6=df[df["Class"]==6]
sub10=df[df["Class"]==10]
sub15=df[df["Class"]==15]
sub16=df[df["Class"]==16]
sub20=df[df["Class"]==20]
sub21=df[df["Class"]==21]

In [11]:
index6=sub6.index.tolist()
index6

[166]

In [12]:
index10=sub10.index.tolist()
index10

[187, 188]

In [13]:
index15=sub15.index.tolist()
index15

[264, 265]

In [14]:
index16=sub16.index.tolist()
index16

[266]

In [15]:
index20=sub20.index.tolist()
index20

[312, 313]

In [16]:
index21=sub21.index.tolist()
index21

[314]

In [17]:
indices=index6+index10+index15+index16+index20+index21
indices

[166, 187, 188, 264, 265, 266, 312, 313, 314]

In [18]:
df1=df.drop(index=indices)
df1.shape

(330, 16)

In [19]:
y=df1[["Class"]]
y.groupby("Class")["Class"].count()

Class
1     84
2     20
3      9
4     14
5     39
7     14
8      6
11    28
12    16
13     7
14    24
17    10
18    29
19     6
22    24
Name: Class, dtype: int64

In [20]:
X=df1.drop(["Class"],axis=1)
X.head()

,1,2,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,1.0,2,2,1,2,2,2,2,2.0,2,2,2.0,2,2
1,1,1.0,2,2,2,2,2,1,2,2.0,2,1,2.0,1,2
2,1,2.0,1,2,2,2,2,2,2,2.0,2,2,2.0,1,2
3,1,2.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2
4,1,2.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2


In [21]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
12,26.302599
10,23.468409
1,11.183648
6,7.805680
7,7.007107
13,6.978592
2,6.800290
14,5.396885
11,3.251895
0,2.863530


In [22]:
X_scores=scores[scores["scores"]>6]
len(X_scores)

7

In [22]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,15,13,2,9,10,16,5
0,2.0,2,1.0,2,2,2,2
1,2.0,2,1.0,2,1,1,2
2,2.0,2,2.0,2,2,1,1
3,2.0,2,2.0,2,2,1,1
4,2.0,2,2.0,2,2,1,1


In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(Xp,y,test_size=0.303,random_state=42)
print(X_train.shape)
print(X_test.shape)

(230, 7)
(100, 7)


In [24]:
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(k_neighbors=2,random_state=42)
#X_res, y_res = sm.fit_resample(X_train, y_train)

In [25]:
#print(X_res.shape)

In [26]:
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
model1=OneVsRestClassifier(GaussianNB()).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
model1

OneVsRestClassifier(estimator=GaussianNB())

In [27]:
from sklearn.metrics import accuracy_score
train_pred=model1.predict(X_train)
test_pred=model1.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.2826086956521739
Accuracy for test data:  0.25


In [28]:
from sklearn.naive_bayes import BernoulliNB
model2=OneVsRestClassifier(BernoulliNB()).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
model2

OneVsRestClassifier(estimator=BernoulliNB())

In [29]:
train_pred=model2.predict(X_train)
test_pred=model2.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.24347826086956523
Accuracy for test data:  0.28


In [30]:
from sklearn.naive_bayes import MultinomialNB
model3=OneVsRestClassifier(MultinomialNB()).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
model3

OneVsRestClassifier(estimator=MultinomialNB())

In [31]:
train_pred=model3.predict(X_train)
test_pred=model3.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.24347826086956523
Accuracy for test data:  0.28


In [32]:
from sklearn.neighbors import KNeighborsClassifier
model4=OneVsRestClassifier(KNeighborsClassifier(n_neighbors=1)).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
model4

OneVsRestClassifier(estimator=KNeighborsClassifier(n_neighbors=1))

In [33]:
train_pred=model4.predict(X_train)
test_pred=model4.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.4956521739130435
Accuracy for test data:  0.42


In [34]:
from sklearn.multiclass import OneVsOneClassifier
model5=OneVsOneClassifier(KNeighborsClassifier(n_neighbors=1)).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
model5

OneVsOneClassifier(estimator=KNeighborsClassifier(n_neighbors=1))

In [35]:
train_pred=model5.predict(X_train)
test_pred=model5.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.4608695652173913
Accuracy for test data:  0.3


In [36]:
model6=KNeighborsClassifier(n_neighbors=1).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
model6

KNeighborsClassifier(n_neighbors=1)

In [37]:
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.4956521739130435
Accuracy for test data:  0.42


In [38]:
model6=KNeighborsClassifier(n_neighbors=2).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.4391304347826087
Accuracy for test data:  0.36


In [39]:
model6=KNeighborsClassifier(n_neighbors=3).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.44782608695652176
Accuracy for test data:  0.43


In [40]:
model6=KNeighborsClassifier(n_neighbors=4).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.4652173913043478
Accuracy for test data:  0.45


In [41]:
model6=KNeighborsClassifier(n_neighbors=5).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.45217391304347826
Accuracy for test data:  0.37


In [42]:
model6=KNeighborsClassifier(n_neighbors=6).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.45652173913043476
Accuracy for test data:  0.39


In [43]:
model6=KNeighborsClassifier(n_neighbors=7).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.45217391304347826
Accuracy for test data:  0.37


In [44]:
model6=KNeighborsClassifier(n_neighbors=8).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.4608695652173913
Accuracy for test data:  0.38


In [45]:
model6=KNeighborsClassifier(n_neighbors=9).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.4434782608695652
Accuracy for test data:  0.36


In [46]:
model6=KNeighborsClassifier(n_neighbors=10).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model6.predict(X_train)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.45652173913043476
Accuracy for test data:  0.34


In [47]:
from sklearn import svm
model7=svm.SVC(random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model7.predict(X_train)
test_pred=model7.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.49130434782608695
Accuracy for test data:  0.46


In [48]:
from sklearn.ensemble import RandomForestClassifier
model8=RandomForestClassifier(n_estimators=200,min_samples_split=3,
                              random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model8.predict(X_train)
test_pred=model8.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.5304347826086957
Accuracy for test data:  0.49


In [49]:
from sklearn.ensemble import GradientBoostingClassifier
model9=GradientBoostingClassifier(n_estimators=120,
                              random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model9.predict(X_train)
test_pred=model9.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.5304347826086957
Accuracy for test data:  0.48


#### The most promising algorithm is RandomForestClassifier. Let's see how RandomForest would have performed without feature selection.

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.303,random_state=42)
print(X_train.shape)
print(X_test.shape)

(230, 15)
(100, 15)


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model8=RandomForestClassifier(n_estimators=200,min_samples_split=3,
                              random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model8.predict(X_train)
test_pred=model8.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.8
Accuracy for test data:  0.44


#### Without feature selection, accuracy on train data is much higher. However, accuracy on test data is lower.

In [30]:
X=df.drop(["Class"],axis=1)
y=df[["Class"]]

In [31]:
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
12,18.688902
10,17.782027
1,8.563579
6,6.094379
2,5.378553
7,5.234915
13,5.015078
14,4.266184
11,2.888468
4,2.573905


In [32]:
X_scores=scores[scores["scores"]>5]
len(X_scores)

7

In [33]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,15,13,2,9,5,10,16
0,2.0,2,1.0,2,2,2,2
1,2.0,2,1.0,2,2,1,1
2,2.0,2,2.0,2,1,2,1
3,2.0,2,2.0,2,1,2,1
4,2.0,2,2.0,2,1,2,1


In [173]:
X_train,X_test,y_train,y_test=train_test_split(Xp,y,test_size=0.3,random_state=42)
print(X_train.shape)
print(X_test.shape)

(237, 7)
(102, 7)


In [174]:
model10=RandomForestClassifier(n_estimators=30,bootstrap=False,
                              random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model10.predict(X_train)
test_pred=model10.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.5443037974683544
Accuracy for test data:  0.4117647058823529


In [175]:
model11=GradientBoostingClassifier(n_estimators=100,learning_rate=0.2,
                              random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model11.predict(X_train)
test_pred=model11.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.5189873417721519
Accuracy for test data:  0.47058823529411764


In [176]:
model12=svm.SVC(C=2,random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model12.predict(X_train)
test_pred=model12.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.5189873417721519
Accuracy for test data:  0.46078431372549017


In [34]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
print(X_train.shape)
print(X_test.shape)

(237, 15)
(102, 15)


In [36]:
from sklearn.ensemble import GradientBoostingClassifier
model11=GradientBoostingClassifier(n_estimators=100,learning_rate=0.2,
                              random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model11.predict(X_train)
test_pred=model11.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.8059071729957806
Accuracy for test data:  0.37254901960784315


#### When the extreme minority classes, classes with only 1 or 2 instances were kept, the most promising algorithm is GradientBoostingClassifier. Feature selection prevented overfitting as well. 

#### Comparison of three most promising algorithms for the "primary tumour dataset":

|Model|Extreme minority classes were removed||Extreme minority classes were kept||
|---|---|---|--|--|
||Train accuracy|Test accuracy|Train accuracy|Test accuracy|
|RandomForestClassifier|0.53|0.49|0.544|0.412|
|GradientBoostingClassifier|0.53|0.48|0.52|0.47|
|Support Vector Machine (SVM)|0.49|0.46|0.51|0.46|

#### I implemented feature selection when the extreme minority classes were removed or kept. In  both cases, feature selection prevented overfitting. When the extreme minority classes were removed, the most promising algorithm was RandomForestClassifier, with feature selection, as shown above, train accuracy was 53% and test accuracy was 49%. Without feature selection, train accuray leapt to 80%, however, test accuracy shrank to 44%. When the extreme minority classes were kept, the most promising algorithm was GradientBoostingClassifer, with feature selection, as shown, above, train accuracy was 52% and test accuracy ws 47%. Without feature selection, train accuracy leapt to 80.6%, but test accuracy shrank to 37.3%. For this data, feature selection prevented overfitting.